In [25]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
# Read necessary csv files
df_airports = pd.read_csv("airports.csv")
df_airlines = pd.read_csv("airlines.csv")
df_flights = pd.read_csv("flights.csv")

/Users/user/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
# Build a function to aggerate a new column later
def change_to_rate(n):
    if n > 0 and n <= 599:
        return "0-6AM Rate"
    elif n >= 600 and n <= 1159:
        return "6AM-12PM Rate"
    elif n >= 1200 and n <= 1759:
        return "12-6PM Rate"
    else:
        return "6PM-12PM Rate"

In [28]:
# Add a column which seperate the departure time to differenct sections
df_flights["RATE"] = df_flights["DEPARTURE_TIME"].apply(change_to_rate)

In [29]:
# Get the number of total flights by sessions
df_1 = df_flights[df_flights["CANCELLED"] == 0].groupby(["ORIGIN_AIRPORT", "RATE"]).agg(TOTAL_FLIGHTS_BY_SESSIONS = ("RATE", "count"))

In [30]:
# Get the number of total flights by airports
df_2 = df_flights[df_flights["CANCELLED"] == 0].groupby("ORIGIN_AIRPORT").agg(TOTAL_FLIGHTS_BY_AIRPORTS = ("RATE", "count"))

In [31]:
# Merge the dataframes
df_result = df_1.reset_index().merge(df_2.reset_index(), on = "ORIGIN_AIRPORT", how = "inner")
# Add a column to calculate the ratio per sessions
df_result["RATIO"] = df_result["TOTAL_FLIGHTS_BY_SESSIONS"] / df_result["TOTAL_FLIGHTS_BY_AIRPORTS"] * 100

In [32]:
# As some of the airport names are valid, filter those by airports.csv
df_filtered = df_result.merge(df_airports, left_on = "ORIGIN_AIRPORT", right_on = "IATA_CODE", how = "inner")
df_filtered = df_filtered[["ORIGIN_AIRPORT", "RATE", "RATIO"]]

In [33]:
# Reshape it as a pivot table for easier to read
df_pivot = df_filtered[["ORIGIN_AIRPORT", "RATE", "RATIO"]]
df_pivot = df_pivot.pivot(index = "ORIGIN_AIRPORT", columns = "RATE", values = "RATIO")

In [35]:
df_pivot.head(10)

RATE,0-6AM Rate,12-6PM Rate,6AM-12PM Rate,6PM-12PM Rate
ORIGIN_AIRPORT,,,,
ABE,11.722595,46.935123,37.807606,3.534676
ABI,12.275986,26.075269,50.492832,11.155914
ABQ,6.275026,41.517387,37.839831,14.367756
ABR,46.304676,49.622926,3.167421,0.904977
ABY,24.913495,47.404844,23.068051,4.613610
ACK,NaN,72.016461,19.135802,8.847737
ACT,17.478882,37.166992,26.055880,19.298246
ACV,18.991332,38.140268,24.901497,17.966903
ACY,5.294451,31.823330,45.753114,17.129105
